<a href="https://colab.research.google.com/github/Swastikataria/ExpiryDateDetection/blob/main/EXPIRY_DATE_EXTRACTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install tesseract-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [ ]:
!pip install opencv-python-headless pytesseract numpy


In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract
!pip install opencv-python-headless


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [ ]:
import cv2
import pytesseract
import re
import pandas as pd
from datetime import datetime
from PIL import Image
import io
import numpy as np
from google.colab import files

# Upload images (you've already done this step)
uploaded = files.upload()

# List of keywords related to expiry or manufacturing dates
keywords = [
    "best before", "expiry date", "exp. date", "use by", "sell by", "Exp. date",
    "best-by", "mfg date", "packed on", "prod date", "EXP.", "Best before", "EXP",
    "Exp. Date", "EXP. date", "MFD.", "BEST BEFORE", "MFG.", "MFG Date", "MFG date",
    "EXP:", "Best Before :", "EXP", "USE BY", "Packed on", "Mfg. Date", "Best before","BB"
]

# Regex patterns for various date formats
standard_date_pattern = r"\b(\d{1,2}[./-]\d{1,2}[./-]\d{2,4})\b"  # Handles dd/mm/yyyy and similar
text_date_pattern = r"\b(\d{1,2}[-\s][A-Z]{3}[-\s]\d{4})\b"  # Handles "14 Oct 2020"
ambiguous_date_pattern = r"\b(\d{6})\b"  # Handles dates like "040917"
compact_date_pattern = r"\b(\d{8})\b"  # Handles compact dates like "20260326"
month_year_pattern = r"\b(\d{1,2}[-/.]\d{2,4})\b"  # Handles month-year format like "10/2020"

# Function to handle 6-digit ambiguous dates (e.g., '040917' -> '04-09-2017' or '17-09-2004')
def convert_ambiguous_date(date_str):
    try:
        date1 = datetime.strptime(date_str, "%d%m%y").strftime("%Y-%m-%d")
        date2 = datetime.strptime(date_str, "%y%m%d").strftime("%Y-%m-%d")
        return date1  # Choosing the first interpretation for simplicity
    except ValueError:
        return "Invalid Date"

# Function to handle 8-digit compact dates (e.g., '20260326')
def convert_compact_date(date_str):
    try:
        return datetime.strptime(date_str, "%Y%m%d").strftime("%Y-%m-%d")
    except ValueError:
        return "Invalid Date"

# Function to parse text-based dates like '14 Oct 2020' or '14-Oct-2020'
def parse_text_date(date_str):
    date_str = date_str.replace("-", " ")  # Normalize format
    try:
        return datetime.strptime(date_str, "%d %b %Y").strftime("%Y-%m-%d")
    except ValueError:
        return "Invalid Date"

# Function to normalize standard date formats like "14/10/2020", "14.10.2020", etc.
def normalize_standard_date(date_str):
    try:
        # Handle dates like '14/10/2020', '14-10-2020', '14.10.2020'
        for fmt in ("%d/%m/%Y", "%d-%m-%Y", "%d.%m.%Y", "%d/%m/%y", "%d-%m-%y", "%d.%m.%y"):
            try:
                return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d")
            except ValueError:
                continue
        return "Invalid Date"
    except ValueError:
        return "Invalid Date"

# Function to calculate the difference in days between the current date and the expiry date
def days_remaining(expiry_date_str):
    try:
        expiry_date = datetime.strptime(expiry_date_str, "%Y-%m-%d")
        current_date = datetime.now()
        remaining_days = (expiry_date - current_date).days
        return remaining_days if remaining_days >= 0 else "Expired"
    except ValueError:
        return "Invalid Date"

# Extract dates from the uploaded images
def extract_dates_from_images(uploaded_files):
    results = []

    for filename, file_data in uploaded_files.items():
        print(f"Processing: {filename}")

        # Read the image from uploaded files
        image = Image.open(io.BytesIO(file_data))
        img = np.array(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Extract text using Tesseract
        text = pytesseract.image_to_string(gray)

        # Detect keywords and date patterns
        found_keywords = []
        all_dates = []
        for kw in keywords:
            if kw.lower() in text.lower():
                found_keywords.append(kw.lower())  # Normalize keyword to lowercase
            if len(all_dates) >= 2:
                break  # Stop iteration once two dates are found

        # Find dates using regex patterns
        standard_dates = re.findall(standard_date_pattern, text)
        text_dates = re.findall(text_date_pattern, text)
        ambiguous_dates = re.findall(ambiguous_date_pattern, text)
        compact_dates = re.findall(compact_date_pattern, text)
        month_year_dates = re.findall(month_year_pattern, text)

        # Convert detected dates
        normalized_standard_dates = [normalize_standard_date(d) for d in standard_dates]
        parsed_text_dates = [parse_text_date(d) for d in text_dates]
        converted_ambiguous_dates = [convert_ambiguous_date(d) for d in ambiguous_dates]
        converted_compact_dates = [convert_compact_date(d) for d in compact_dates]

        # Collect all detected dates
        all_dates = (
            normalized_standard_dates +
            parsed_text_dates +
            converted_ambiguous_dates +
            converted_compact_dates +
            month_year_dates
        )

        # Filter out invalid dates
        all_dates = [d for d in all_dates if d != "Invalid Date"]

        if len(all_dates) >= 2:
            all_dates = all_dates[:2]  # Stop after capturing two dates

        # Calculate days remaining if an expiry date is detected
        if all_dates:
            expiry_date = all_dates[0]  # Assume first detected date is the expiry date
            days_left = days_remaining(expiry_date)
        else:
            days_left = "N/A"

        # Save results for this image
        results.append({
            "Image": filename,
            "Detected Keywords": ", ".join(set(found_keywords)) if found_keywords else "N/A",
            "Detected Dates": ", ".join(all_dates) if all_dates else "N/A",
            "Days Remaining": days_left
        })

        # Optionally, display or save the marked image
        data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
        for i, word in enumerate(data['text']):
            if any(kw in word.lower() for kw in keywords) or \
               re.match(standard_date_pattern, word) or \
               re.match(text_date_pattern, word) or \
               re.match(ambiguous_date_pattern, word) or \
               re.match(compact_date_pattern, word):
                x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Optionally, display or save the marked image
        marked_image = Image.fromarray(img)
        marked_image.show()  # Displays the image inline in the notebook

    # Create DataFrame from results
    df = pd.DataFrame(results)
    print(df)
    return df

# Example usage
df = extract_dates_from_images(uploaded)


Saving WhatsApp Image 2024-10-19 at 10.39.28 PM.jpeg to WhatsApp Image 2024-10-19 at 10.39.28 PM.jpeg
Saving WhatsApp Image 2024-10-19 at 10.39.30 PM (1).jpeg to WhatsApp Image 2024-10-19 at 10.39.30 PM (1).jpeg
Saving WhatsApp Image 2024-10-19 at 10.39.27 PM.jpeg to WhatsApp Image 2024-10-19 at 10.39.27 PM.jpeg
Saving WhatsApp Image 2024-10-19 at 10.39.27 PM (1).jpeg to WhatsApp Image 2024-10-19 at 10.39.27 PM (1).jpeg
Saving WhatsApp Image 2024-10-19 at 10.39.27 PM (2).jpeg to WhatsApp Image 2024-10-19 at 10.39.27 PM (2).jpeg
Saving WhatsApp Image 2024-10-19 at 10.39.26 PM.jpeg to WhatsApp Image 2024-10-19 at 10.39.26 PM.jpeg
Saving WhatsApp Image 2024-10-18 at 11.26.13 PM.jpeg to WhatsApp Image 2024-10-18 at 11.26.13 PM.jpeg
Saving WhatsApp Image 2024-10-18 at 11.26.15 PM.jpeg to WhatsApp Image 2024-10-18 at 11.26.15 PM.jpeg
Saving WhatsApp Image 2024-10-18 at 11.26.13 PM (1).jpeg to WhatsApp Image 2024-10-18 at 11.26.13 PM (1).jpeg
Saving WhatsApp Image 2024-10-18 at 11.26.11 PM (1